<a href="https://colab.research.google.com/github/aritejhg/tinyyolov4_person_detector/blob/main/Door_detector_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#collins people at door detector

Objectives:
1. Convert Sunmi and Top View Multiple Person dataset to YOLO format
2. Implement histogram equalisation
3. Get baseline from KDE background subtraction + Historgram
4. Train TinyYOLOv4 model and check performance. Implement with DIoU + Multi-Scale

Result required: Just a indication if people are present in the area. To be passed to other code for talking to MiR



References:
1. [Training tinyYOLOv4](https://colab.research.google.com/drive/1hQO4nOoD6RDxdbz3C1YSiifTsyZjZpYm?usp=sharing)
2. [Top View Multi Person labelled images zip](https://drive.google.com/drive/folders/1dc1QrDvZPWXbUfzEQE5EPIAz2CPCVI38)
3. [Sunmi dataset](https://github.com/Sunmi-AI-Lab/head-detection-and-tracking/tree/master/datasets) 
4. [DIoU](https://arxiv.org/abs/1911.08287)
5. [Multi-Scale tinyYOLOv4 Research paper](https://link.springer.com/chapter/10.1007/978-3-030-85383-9_1)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
!rm -r /content/__MACOSX/


# Download datasets

##Sunmi dataset

In [ ]:
!apt install subversion

In [ ]:
!svn checkout https://github.com/Sunmi-AI-Lab/head-detection-and-tracking/trunk/datasets/in-office /content/Sunmi/


## Top View 


In [ ]:
!pip install gdown

In [6]:
import os
os.makedirs("/content/TVMP")

In [ ]:
import gdown
#https://drive.google.com/drive/folders/1dc1QrDvZPWXbUfzEQE5EPIAz2CPCVI38?usp=sharing
!gdown --folder https://drive.google.com/drive/folders/1dc1QrDvZPWXbUfzEQE5EPIAz2CPCVI38?usp=sharing /content/TVMP/

In [ ]:
!unzip /content/TVMP/labeled_images/test.zip

In [ ]:
!unzip /content/TVMP/labeled_images/train.zip 